In [1]:
import os
from langchain.chat_models import init_chat_model
llm = init_chat_model("groq:llama-3.1-8b-instant")

/Users/manasvi/Github/Agents-Langchain-Langraph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x14196ee10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x141809d30>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain.tools import tool

@tool 
def create_calendar_event(
    name: str,
    start_date: str,
    end_date: str,
    description: str,
    attendees: list[str]
) -> str:
    """Create a calendar event"""
    return f"Calendar event {name} created from {start_date} to {end_date} with description {description} and attendees {attendees}"


@tool
def send_email(
    to: str,
    subject: str,
    body: str
) -> str:
    """Send an email"""
    return f"Email sent to {to} with subject {subject}"


@tool
def get_available_slots(
    attendees: list[str],
    date: str,
    duration: int
) -> list[str]:
    """Get available slots for a given date and duration"""
    return f"Available slots for {date} with duration {duration} "



In [12]:
from langchain.agents import create_agent

calendar_prompt = """
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
 """

tools = [create_calendar_event, get_available_slots]

calendar_agent = create_agent(llm, tools, system_prompt=calendar_prompt)

query = " schedule a meeting this wednesday at 2 pm for 1 hour "
calendar_agent.invoke({"input": query})

# for event in calendar_agent.stream(
#     {"messages": [{"role": "user", "content": query}]},
#     stream_mode="values",
# ):
#     event["messages"][-1].pretty_print()
for step in calendar_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()


================================== Ai Message ==================================
Tool Calls:
  get_available_slots (pd361e0mx)
 Call ID: pd361e0mx
  Args:
    attendees: []
    date: this week
    duration: 60
  create_calendar_event (gqdts835h)
 Call ID: gqdts835h
  Args:
    attendees: []
    description: 
    end_date: 2024-01-01T15:00:00
    name: Meeting
    start_date: 2024-01-01T14:00:00
================================= Tool Message =================================
Name: get_available_slots

Available slots for this week with duration 60 
================================= Tool Message =================================
Name: create_calendar_event

Calendar event Meeting created from 2024-01-01T14:00:00 to 2024-01-01T15:00:00 with description  and attendees []
================================== Ai Message ==================================

To schedule a meeting, I need to first determine the available time slots for this week with a duration of 1 hour. I will then use this info

In [8]:
from langchain.agents import create_agent


email_prompt = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)


email_agent = create_agent(llm, tools = [send_email], system_prompt=email_prompt)

query = " send an email to John about the postponing the meeting to next week "
email_agent.invoke({"input": query})

# for event in email_agent.stream(
#     {"messages": [{"role": "user", "content": query}]},
#     stream_mode="values",
# ):
#     event["messages"][-1].pretty_print()

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  send_email (dd49x1csc)
 Call ID: dd49x1csc
  Args:
    body: Dear John, I wanted to reach out to you regarding the meeting that was scheduled for this week. Due to unforeseen circumstances, I am writing to inform you that it has been postponed to next week. I will be in touch with a new set of dates and times shortly. Thank you for your understanding.
    subject: Postponement of Meeting
    to: john@example.com
================================= Tool Message =================================
Name: send_email

Email sent to john@example.com with subject Postponement of Meeting
================================== Ai Message ==================================

"The email was sent to john@example.com with the subject Postponement of Meeting and a message informing John that the meeting has been postponed to next week."


### Wrap sub-agents as tools

In [9]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

### supervisor agent

In [15]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
    "dont call multiple tools for the simple task only call multiple tools if the task is complex"
)

supervisor_agent = create_agent(
    llm,
    tools=[schedule_event, manage_email],
    system_prompt=SUPERVISOR_PROMPT,
)

In [17]:
query = "Schedule a meeting with John on Tuesday at 2pm"

for step in supervisor_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (xdg77x2v9)
 Call ID: xdg77x2v9
  Args:
    request: meeting with John on Tuesday at 2pm
================================= Tool Message =================================
Name: schedule_event

I scheduled a meeting with John from 2pm to 3pm on Tuesday, January 3rd, 2024. The available slots for that day with a 60-minute duration are:


BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: attempted to call tool 'brave_search' which was not in request.tools", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=brave_search>{"query": "Tuesday January 3rd 2024 available 2pm 60 minutes"}</function>'}}

In [ ]:
query = "Send the design team a reminder about reviewing the new mockups"

for step in email_agent.stream(
    {"messages": [{"role": "user", "content": query}]}
):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()